## Part 2: PyTorch Implementation

1. Importing basic libraries and setting device to GPU

In [3]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, auc, confusion_matrix, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. Loading Dataset

In [ ]:
df = pd.read_csv('data.csv')
X = df.drop(columns=['no_show_bin']).astype(np.float32).values
y = df['no_show_bin'].astype(np.float32).values.reshape(-1, 1)

3. Scaling numerical features

In [5]:
continuous_cols = ['age', 'wait_days', 'sched_hour','past_appointments', 'patient_no_show_rate']
target_col = 'no_show_bin'
binary_cols = [col for col in df.columns if col not in continuous_cols + [target_col]]
X_continuous = df[continuous_cols]
X_binary = df[binary_cols].astype(np.float32)
y = df[target_col].astype(np.float32).values.reshape(-1, 1)
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous)
X = np.hstack([X_continuous_scaled, X_binary.values])

4. Train Test Split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

5. Custom Dataset Class

In [7]:
class NoShow(Dataset):
  def __init__(self, X, y):
    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y)
  def __len__(self):
    return len(self.X)
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

6. Creating Dataloaders

In [8]:
train = NoShow(X_train, y_train)
test = NoShow(X_test, y_test)
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

7. Neural Network

In [9]:
class NeuralNetworkPytorch(nn.Module):
  def __init__(self,input_size):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(input_size, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid()
    )

  def forward(self,x):
    return self.network(x)

8. Training Function

In [10]:
def train_model(model, train_loader, criterion, optimizer, epochs=100,verbose=True):
  for epoch in range(epochs):
    model.train()
    losses = []
    for X,y in train_loader:
      X, y = X.to(device), y.to(device)
      optimizer.zero_grad()
      output = model(X)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
      losses.append(loss.item())
    if(epoch%10==0 and verbose):
      print(f'Epoch: {epoch}, Loss: {np.mean(losses)}')

9. Training the model on dataset

In [13]:
input_size = X_train.shape[1]
model = NeuralNetworkPytorch(input_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

start = time.time()
train_model(model, train_loader, criterion, optimizer, epochs=10, verbose=False)
end = time.time()

10. Evaluation Metrics

In [14]:
y_p = model.forward(test.X.to(device))
y_p = y_p.detach().cpu().numpy()
y_pred = (y_p > 0.5).astype(int)

print("Training time:",end-start)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("F1-Score:",f1_score(y_test, y_pred))
print("PR-AUC: ",average_precision_score(y_test,y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training time: 26.567100048065186
Test Accuracy: 0.9158975750995295
F1-Score: 0.7810623012601579
PR-AUC:  0.6610364078782504
Confusion Matrix:
 [[16929   755]
 [ 1104  3316]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.96      0.95     17684
         1.0       0.81      0.75      0.78      4420

    accuracy                           0.92     22104
   macro avg       0.88      0.85      0.86     22104
weighted avg       0.91      0.92      0.91     22104

